In [64]:
import re
import json
import requests
import dateparser
import pandas as pd
from bs4            import BeautifulSoup
from pprint         import pprint
from IPython.display import Image
from IPython.core.display import HTML
from IPython import display

- no partner?
- doing hw6 as practice?

In [55]:
def get_id(url):
    id = ''

    # Regex find ID
    match = re.search(r'\/(area|route)\/(\d+)', url)

    # Check valid match
    if match:
        # Extract the ID number from the first capturing group
        id = match.group(2)

    # Invalid match
    else:
        raise Exception('Unable to locate route/area ID in url - {url}')
    
    return id

def get_directory():
    # Make request to get the route guide page
    url     = 'https://www.mountainproject.com/route-guide'
    content = requests.get(url).content
    soup    = BeautifulSoup(content, features = 'html.parser')
    guide   = soup.find(id = 'route-guide')

    # Create a dictionary to store the directory
    directory = {}

    # Get the list of areas
    areas = [strong.find('a') for strong in guide.find_all('strong')]

    for area in areas:
        # Get the area name
        name = area.text

        # Get the area link
        link = area['href']

        # Save to directory
        directory[name] = link

    return directory

# Gets an area from MP given its ID
def get_area(id):
    url     = f'https://www.mountainproject.com/api/v2/areas/{id}'
    data = requests.get(url).json()
    return data

# Gets a list of photos of the first few pics of a route
def get_photos(route_id):
    url = f'https://www.mountainproject.com/route/{route_id}'
    content = requests.get(url).content
    soup = BeautifulSoup(content, features = 'html.parser')

    photos = soup.find_all('img')
    photos = filter(lambda x: 'data-src' in x.attrs, photos)
    photos = map(lambda x: { 'url': x['data-src'].replace('smallMed', 'medium'), 'alt': x['alt'] }, photos)

    return list(photos)

# Gets a route from MP given its ID
def get_route(id):
    url     = f'https://www.mountainproject.com/api/v2/routes/{id}'
    data    = requests.get(url).json()
    data['photos'] = get_photos(id)
    return data

def get_routes_in(area_id, route_list, only_ids = True):
    area = get_area(area_id)

    for i, child in enumerate(area['children']):
        child_id = str(child['id'])

        if child['type'] == 'Route':
            if only_ids:
                route_list.append(child_id)
            else:
                route = get_route(child_id)
                route_list.append(route)
        else:
            get_routes_in(child_id, route_list, only_ids)

# DANGER: this function might take a couple billion years to run
def get_all_routes_ids():
    directory = get_directory()
    del directory['International']

    route_list = []

    for state in directory:
        area_id = get_id(directory[state])
        get_routes_in(area_id, route_list)

    return route_list

In [73]:
# Get all routes in an area
area_id = '105874223'
waimea_routes = []
get_routes_in(area_id, waimea_routes, only_ids = False)

In [76]:
for route in waimea_routes:
  if len(route['photos']) > 0:
    print(f'{route['title']}, {route['difficulty']}')

    # Display the first photo
    display.display(HTML(f'<img src="{route["photos"][0]["url"]}">'))
    # Image(url = route['photos'][0]['url'])

Ali Babbler, 5.11c


Little Big Man, 5.11b


Supersize Me, 5.6


Working Man, 5.7


Fat Man, 5.7


...

Goldbug, 5.10d


Gold Digger, 5.8+


Rock de Jours, 5.9 X


Rock Du Jours Direct, 5.9+


Going Nuts, 5.13b
